In [3]:
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors

In [6]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
    --------------------------------------- 0.5/38.8 MB 728.2 kB/s eta 0:00:53
    --------------------------------------- 0.5/38.8 MB 728.2 kB/s eta 0:00:53
    --------------------------------------- 0.8/38.8 MB 744.7 kB/s eta 0:00:52
   - -------------------------------------- 1.0/38.8 MB 739.8 kB/s eta 0:00:52
   - -------------------------------------- 1.0/38.8 MB 739.8 kB/s eta 0:00:52
   - -------------------------------------- 1.3/38.8 MB 745.3 kB/s eta 0:00:51
   - -------------------------------------- 1.3/38.8 MB 745.3 kB/s eta 0:00:51
   - -------------------------------------- 1.6/38.8 MB 742.2 kB/s eta 0:00:51
   - -------------------------------------- 1.8/38.8 MB 740.2 kB/s eta 0:00:50
   - ----

In [7]:
import cv2

In [8]:
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

In [9]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

In [10]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [16]:
img = image.load_img('testing/watch.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


In [17]:
neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [18]:
distances,indices = neighbors.kneighbors([normalized_result])
print(indices)

[[33421 23347   944 18344   970 29605]]


In [20]:
for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)